In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib. pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import random

In [12]:
torch.randint(0, 256, (2, 3, 4, 5), dtype=torch.float32) # 2 images, every image involves 3 channels, 4x5 pixels, every pixel's intensity is in 0;255

tensor([[[[114.,   9., 128.,  84.,  72.],
          [214.,  41.,  28., 126., 220.],
          [137.,  83.,   4., 179., 235.],
          [129., 192., 193., 177., 170.]],

         [[175.,  18., 220., 172., 192.],
          [149., 219., 170., 182., 163.],
          [219., 206., 125., 207., 107.],
          [213.,  68.,  72., 131.,  42.]],

         [[ 41.,  10.,  16.,  48., 176.],
          [160., 133., 213., 121., 172.],
          [ 50., 118., 134., 138.,   2.],
          [ 91.,  29.,  12., 108., 234.]]],


        [[[130., 187., 162., 220., 213.],
          [202.,  95., 198.,  77.,  53.],
          [248., 191.,  28.,  72., 125.],
          [ 90.,  54., 136., 148., 185.]],

         [[ 96.,   7.,  48.,  64., 173.],
          [ 29.,  63., 163.,   2.,  88.],
          [164., 101., 123., 133., 208.],
          [ 76.,  25., 171., 166.,  88.]],

         [[ 18., 161.,  24.,  79., 157.],
          [ 64., 113., 150., 173.,  63.],
          [151., 138., 200., 108.,   9.],
          [ 29., 146.,

In [13]:
torch.cuda.is_available()

False

In [43]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [20]:
train_dataset = datasets.MNIST(root = './mnist', train=True, download=True, transform=transforms.ToTensor())
test_dataset = datasets.MNIST(root = './mnist', train=False, download=True, transform=transforms.ToTensor())

In [23]:
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

In [ ]:
len(next(iter(train_loader)))  # list including 2 items, images and numbers(label)

2

In [34]:
next(iter(train_loader))[0].shape

torch.Size([64, 1, 28, 28])

In [36]:
next(iter(train_loader))[1]

tensor([4, 1, 2, 0, 6, 1, 6, 9, 1, 3, 3, 2, 4, 6, 4, 9, 1, 5, 5, 1, 7, 8, 2, 2,
        0, 4, 4, 4, 6, 3, 6, 5, 8, 3, 8, 3, 6, 9, 0, 8, 0, 6, 2, 3, 2, 6, 8, 2,
        3, 8, 1, 6, 7, 4, 4, 8, 2, 6, 0, 8, 0, 7, 4, 9])

In [ ]:
class OddiyCNN(nn.Module):
    def __init__(self):
        super().__init__()

        self.net = nn.Sequential(
            # 2 blocks with Conv, Relu, Maxpool
            nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1),    # 1 channel 28x28 image, 16 3x3 filters, 1 padding to keep orginal size 28x28
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                                  # 2X2 grid with stride=2, reduces 2times size of feature map as 14x14
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1),   # 16 channel 14x14 feature map, 32 3x3 filters, 1 padding to keep orginal size 14x14 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),                                  # 2X2 grid with stride=2, reduces 2times size of feature map as 7x7

            # Flatten, Linear, Relu, Linear
            nn.Flatten(),                                                           # Flatten the feature map (32x7x7 → 1568) for Linear function readable
            nn.Linear(32*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )

    def forward(self, x):
        return self.net(x)

In [66]:
model = OddiyCNN()

In [67]:
criterion = nn.CrossEntropyLoss()
optimiser = optim.Adam(model.parameters(), lr=0.001)

In [68]:
epochs = 10
losses = []
for epoch in range(epochs):
    model.train()
    epoch_loss = 0

    for image, label in train_loader:
        image, label = image.to(device), label.to(device)

        # Forward pass
        output = model(image)
        loss = criterion(output, label)

        # Backward pass
        optimiser.zero_grad()
        loss.backward()
        optimiser.step()

        epoch_loss += loss.item()
    
    losses.append(epoch_loss/len(train_loader))
    print(f'Epoch {epoch+1}, Loss:{epoch_loss/len(train_loader):.4f}')

Epoch 1, Loss:0.2430
Epoch 2, Loss:0.0619
Epoch 3, Loss:0.0425
Epoch 4, Loss:0.0323
Epoch 5, Loss:0.0264
Epoch 6, Loss:0.0213
Epoch 7, Loss:0.0164
Epoch 8, Loss:0.0142
Epoch 9, Loss:0.0109
Epoch 10, Loss:0.0116


In [69]:
model.eval()
total = 0
correct = 0

with torch.no_grad():
    for image, label in test_loader:
        output = model(image)
        predict = output.argmax(dim=1)
        correct += (predict == label).sum()
        total += len(label)

print(f'Test accuracy: {correct/total*100}%')

Test accuracy: 99.0199966430664%


In [71]:
total, correct

(10000, tensor(9902))